In [1]:
import sys
sys.path.insert(0, '/home/ah1114/LanguageOfLife/BioDL')
#and import all the stuff
from data import *
from fastai.callbacks import *
from datetime import datetime 

In [30]:
path = Path('./') 
model_path = Path('/home/ah1114/LanguageOfLife/MifaserClassification/models/mifaserclas_k1')
pretrained_path = Path('/home/ah1114/LanguageOfLife/TaxClassification/models/best_GTDB_k1_1000seq_enc')
log_path = Path('/home/ah1114/LanguageOfLife/MifaserClassification/train_logs/mifaserclas_k1')
vocab_path = Path('/home/ah1114/LanguageOfLife/vocabs')
vocab_name = vocab_path/'ngs_vocab_k1_withspecial.npy'

#data_path = Path('/scratch/ah1114/LoL/data/GTDBrepCDS_chunked_csv/')
data_path = Path('/scratch/ah1114/LoL/data/mifaser_out/Small100_EvenEnvPackages/clean_for_training/')
train_path = Path('/scratch/ah1114/LoL/data/mifaser_out/Small100_EvenEnvPackages/clean_for_training/train/')
valid_path = Path('/scratch/ah1114/LoL/data/mifaser_out/Small100_EvenEnvPackages/clean_for_training/valid/')

lr_plot_out = '/home/ah1114/LanguageOfLife/MifaserClassification/lrplot_mifaserclas_k1.png'
#if stuck with 128GB, might try doing it in shuffled batches of files?
bs=128 #2048 worked in previous tests
ksize=1
stride=1
max_seqs=100
valid_max_seqs=1000000
lrate=1e-3 #adjusted down after seeing lrplot
drop_mult=0.3 #haven't tuned this at all yet
num_cycles=1
numrounds=5
skiprows_file = "skiprows_mifaser.csv"

tok = BioTokenizer(ksize=ksize, stride=stride)
if vocab_name.is_file():
    voc = np.load(vocab_name)
    model_voc = BioVocab(voc)
else:
    model_voc = BioVocab.create_from_ksize(ksize=ksize)
    np.save(vocab_name, model_voc.itos)

In [31]:
data = BioClasDataBunch.from_multiple_csv(path=data_path, train=train_path, valid=valid_path,
                                        text_cols='seq', label_cols='annotation',
                                        classes=list(np.load('mifaser_classes.npy')),
                                        tokenizer=tok, vocab=model_voc,
                                        max_seqs_per_file=max_seqs, valid_max_seqs=valid_max_seqs, skiprows=0, bs=bs
                                            )

You can deactivate this warning by passing `no_check=True`.


/home/ah1114/.pyenv/versions/3.6.9/envs/BioDL/lib/python3.6/site-packages/fastai/basic_data.py:245: UserWarning: Your training dataloader is empty, you have only 100 items in your training set.
                 Your batch size is 128, you should lower it.
  Your batch size is {self.train_dl.batch_size}, you should lower it.""")


In [33]:
data.batch_size

128

In [34]:
config = awd_lstm_clas_config.copy() #make sure this is awd_lstm_clas_config for classification!
config['bidir'] = False #this can be True for classification, but if you're using a pretrained LM you need to keep if = False because otherwise your matrix sizes don't match
learn = text_classifier_learner(data, AWD_LSTM, drop_mult=drop_mult, model_dir=".", config=config, pretrained=False)


In [35]:
learn.load(model_path)

RNNLearner(data=BioClasDataBunch;

Train: LabelList (100 items)
x: BioTextList
xxbos C A A T G A T G T G C C T C A T A T T G G T C A T G C T T A T A C A A C A T T A G C T T G C G A T G T A A T T G C T A G G T T T A A G A G A T T A G A C G A T T T T A A T G T T T T T T T T T T A A C T G,xxbos G T T G T T G A A G T C G T G G G C C A C G C C A C C G G T G A G T T G G C C G A T G G C T T C C A G C T T C T G C G A T T G A C G C A G G G C T T C C T C G G C G G C G G C C A G G G C T T T G G C G C T C T C C C G C T C G G C G G T G A A G T C G C G G C C C A C G G C G T T G A T C T G C T G C T C,xxbos T T A A T C A T G G G A T T A A A A G A T A T T G C T G T T T C A T C T G G T T C A G C A T G T A C T T C T G C A A G T C T T G A G C C A T C A T A T G T T T T A A G A G C A T T G G G C A G A A A A G A,xxbos G T G G A C G G A C G C G T G G T C G A C C A G A T C A A G A T C G C G G T C G A T C A G C T C A A A A A A A A T C C C G A C T C G C G G C G C A T C A T C G T C T C G G C A T G G A A C G T C G

In [10]:
valid = pd.read_csv('/scratch/ah1114/LoL/data/mifaser_out/Small100_EvenEnvPackages/clean_for_training/valid/mifaser_valid.csv',nrows=100)
valid.head()

,run,seq,annotation
0,ERR598955,CTACCAGATTATTTGATAGCTTGCGTTGGAGGTGGATCAAATGCCA...,4.2.1.20
1,ERR3521978,CTGCTAAGGTGTCCATCAACACCAGCTCGGTCCAGTACGAGAACGA...,2.3.1.54
2,ERR2699809,GAACAAATTATGACTAAACAACATAAAAAAGTTATCCTCGTTGGTG...,1.1.1.27
3,ERR2020026,CTTGGGATCCATCAGGCGGATGAGGTCTTTCAGGATGATATTGACG...,1.7.1.13
4,ERR2699809,GAGCCCTTCTTCATGGTGGAGACCATCTCGTCGGTCACGAGCTTCG...,1.4.1.1


In [11]:
interp = TextClassificationInterpretation.from_learner(learn)

In [13]:
for x in range(0,15):
    print('annotation is',valid['annotation'][x])
    
    interp.show_intrinsic_attention(valid['seq'][x])

annotation is 4.2.1.20


annotation is 2.3.1.54


annotation is 1.1.1.27


annotation is 1.7.1.13


annotation is 1.4.1.1


...

annotation is 5.4.99.20


annotation is 2.5.1.31


annotation is 5.99.1.3


annotation is 1.1.1.271


annotation is 2.3.1.54


In [15]:
Counter(valid['annotation']).most_common()

[('2.7.7.6', 8),
 ('6.1.1.6', 3),
 ('4.2.1.20', 2),
 ('2.3.1.54', 2),
 ('5.99.1.3', 2),
 ('6.1.1.3', 2),
 ('1.1.1.42', 2),
 ('2.2.1.1', 2),
 ('3.6.3.54', 2),
 ('2.1.3.9', 2),
 ('1.97.1.12', 2),
 ('3.6.4.12', 2),
 ('2.5.1.54', 2),
 ('6.5.1.2', 2),
 ('1.1.1.27', 1),
 ('1.7.1.13', 1),
 ('1.4.1.1', 1),
 ('1.1.1.86', 1),
 ('2.8.4.4', 1),
 ('6.3.5.7', 1),
 ('2.8.4.5', 1),
 ('5.4.99.20', 1),
 ('2.5.1.31', 1),
 ('1.1.1.271', 1),
 ('3.6.4.13', 1),
 ('3.4.19.5', 1),
 ('2.5.1.7', 1),
 ('2.8.1.7', 1),
 ('6.3.5.2', 1),
 ('1.3.1.86', 1),
 ('1.5.1.2', 1),
 ('2.1.1.13', 1),
 ('3.6.3.12', 1),
 ('4.2.1.59', 1),
 ('6.1.1.15', 1),
 ('2.3.1.29', 1),
 ('1.17.1.2', 1),
 ('4.1.1.49', 1),
 ('4.1.3.1', 1),
 ('4.2.1.115', 1),
 ('6.3.2.8', 1),
 ('3.6.3.14', 1),
 ('1.1.1.95', 1),
 ('2.3.1.41', 1),
 ('3.4.21.92', 1),
 ('2.7.4.22', 1),
 ('4.2.1.3', 1),
 ('3.5.4.5', 1),
 ('5.1.1.13', 1),
 ('1.4.1.16', 1),
 ('1.6.1.2', 1),
 ('2.4.2.29', 1),
 ('2.7.2.3', 1),
 ('3.2.1.131', 1),
 ('2.7.4.3', 1),
 ('3.6.1.23', 1),
 ('1.11

In [18]:
subset = valid[valid['annotation']=='2.7.7.6']
for x in subset.index:
    print('annotation is',valid['annotation'][x])
    
    interp.show_intrinsic_attention(valid['seq'][x])

annotation is 2.7.7.6


annotation is 2.7.7.6


annotation is 2.7.7.6


annotation is 2.7.7.6


annotation is 2.7.7.6


annotation is 2.7.7.6


annotation is 2.7.7.6


annotation is 2.7.7.6


# Some quick figs looking at distributions predicted vs real etc

first pass model got 75% accuracy in second epoch!

In [28]:
valid = pd.read_csv('/scratch/ah1114/LoL/data/mifaser_out/Small100_EvenEnvPackages/clean_for_training/valid/mifaser_valid.csv',nrows=1000000)
valid.head()

,run,seq,annotation
0,ERR598955,CTACCAGATTATTTGATAGCTTGCGTTGGAGGTGGATCAAATGCCA...,4.2.1.20
1,ERR3521978,CTGCTAAGGTGTCCATCAACACCAGCTCGGTCCAGTACGAGAACGA...,2.3.1.54
2,ERR2699809,GAACAAATTATGACTAAACAACATAAAAAAGTTATCCTCGTTGGTG...,1.1.1.27
3,ERR2020026,CTTGGGATCCATCAGGCGGATGAGGTCTTTCAGGATGATATTGACG...,1.7.1.13
4,ERR2699809,GAGCCCTTCTTCATGGTGGAGACCATCTCGTCGGTCACGAGCTTCG...,1.4.1.1


In [29]:
len(Counter(valid['annotation']))
#1274 in total

1242

In [11]:
len(valid)

NameError: name 'valid' is not defined

In [36]:
learn.data.batch_size

128

In [37]:
preds = learn.get_preds()

In [23]:
preds

[tensor([[1.5763e-06, 1.5216e-07, 5.9605e-06,  ..., 1.1116e-04, 1.4315e-06,
          1.7006e-05],
         [1.3639e-08, 6.6665e-10, 1.3972e-08,  ..., 1.5720e-08, 1.8628e-07,
          3.4289e-07],
         [6.7923e-07, 5.4781e-09, 5.9854e-07,  ..., 3.9691e-07, 1.5368e-07,
          9.6131e-07],
         ...,
         [7.0784e-04, 1.5804e-06, 1.0672e-04,  ..., 9.6893e-04, 1.1064e-04,
          1.4500e-04],
         [7.0465e-13, 5.2824e-14, 7.2567e-11,  ..., 8.5059e-12, 3.4915e-14,
          5.0128e-12],
         [6.6495e-05, 2.3962e-06, 1.0661e-04,  ..., 1.4808e-03, 1.5416e-04,
          9.1088e-05]]), tensor([ 581,  931,  435,  ...,  971,  388, 1107])]

In [38]:
preds[0].shape

torch.Size([1000000, 1274])

In [91]:
print(preds[0][0].argmax())
print(preds[1][0])

tensor(392)
tensor(392)


In [97]:
labels = [learn.data.classes[int(x.argmax())] for x in preds[0]]

In [98]:
len(Counter(labels))

768

In [99]:
Counter(labels).most_common()

[('2.7.7.6', 51179),
 ('5.99.1.3', 29815),
 ('3.6.3.14', 26747),
 ('3.6.4.12', 19936),
 ('2.7.13.3', 12502),
 ('4.2.1.11', 10165),
 ('3.6.5.n1', 9170),
 ('6.3.5.3', 8484),
 ('1.10.3.9', 7754),
 ('3.4.21.53', 7739),
 ('2.7.7.7', 7677),
 ('2.7.7.8', 7521),
 ('1.97.1.12', 7222),
 ('1.4.7.1', 7187),
 ('6.2.1.1', 7116),
 ('2.5.1.6', 7104),
 ('6.3.5.2', 6873),
 ('1.11.1.21', 6642),
 ('3.6.3.54', 6615),
 ('2.1.2.1', 6583),
 ('2.1.1.13', 6546),
 ('6.4.1.1', 6408),
 ('4.2.1.3', 6370),
 ('3.6.4.13', 6236),
 ('5.99.1.2', 6027),
 ('4.2.1.20', 5980),
 ('5.3.1.9', 5908),
 ('2.2.1.1', 5760),
 ('6.1.1.7', 5610),
 ('3.3.1.1', 5582),
 ('2.5.1.47', 5574),
 ('6.1.1.3', 5402),
 ('2.6.1.16', 5304),
 ('3.4.21.92', 5289),
 ('6.2.1.5', 5167),
 ('6.3.5.7', 5146),
 ('6.5.1.2', 5090),
 ('1.2.1.12', 5090),
 ('6.1.1.6', 5036),
 ('1.6.1.2', 4897),
 ('1.1.1.205', 4858),
 ('1.17.4.1', 4722),
 ('2.7.7.24', 4655),
 ('2.3.1.179', 4620),
 ('3.2.1.86', 4613),
 ('1.1.1.100', 4567),
 ('2.8.1.7', 4437),
 ('2.3.1.41', 4430),
 

In [100]:
true = [learn.data.classes[x] for x in preds[1]]
print(len(Counter(true)))

1242


In [101]:
predictions = pd.DataFrame({"true":true,"predicted":labels})
predictions.head()

,true,predicted
0,4.2.1.20,4.2.1.20
1,2.3.1.54,2.3.1.54
2,1.1.1.27,1.1.1.27
3,1.7.1.13,1.7.1.13
4,1.4.1.1,2.8.4.3


In [123]:
ones = predictions[predictions['predicted'].str.startswith('1.')]

In [124]:
print(len(ones))
ones.reset_index(inplace=True,drop=True)
ones.head()

165619


,true,predicted
0,1.1.1.27,1.1.1.27
1,1.7.1.13,1.7.1.13
2,1.1.1.86,1.1.1.86
3,1.1.1.271,1.1.1.271
4,1.3.1.86,1.3.1.86


In [128]:
matches = ones[['true', 'predicted']].assign(NE=ones.true != ones.predicted)['NE']
print('accuracy for EC1:',1-np.sum(matches)/len(ones))
pmatches = predictions[['true', 'predicted']].assign(NE=predictions.true != predictions.predicted)['NE']
print('accuracy overall:',1-np.sum(pmatches)/len(predictions))

accuracy for EC1: 0.8380439442334515
accuracy overall: 0.862714


In [125]:
ones['predicted_3'] = ['.'.join(x.split('.')[0:3]) for x in ones['predicted']]
ones['true_3'] = ['.'.join(x.split('.')[0:3]) for x in ones['true']]

/home/ah1114/.pyenv/versions/3.6.9/envs/BioDL/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ah1114/.pyenv/versions/3.6.9/envs/BioDL/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [129]:
ones.head()

,true,predicted,predicted_3,true_3
0,1.1.1.27,1.1.1.27,1.1.1,1.1.1
1,1.7.1.13,1.7.1.13,1.7.1,1.7.1
2,1.1.1.86,1.1.1.86,1.1.1,1.1.1
3,1.1.1.271,1.1.1.271,1.1.1,1.1.1
4,1.3.1.86,1.3.1.86,1.3.1,1.3.1


In [130]:
predictions['predicted_3'] = ['.'.join(x.split('.')[0:3]) for x in predictions['predicted']]
predictions['true_3'] = ['.'.join(x.split('.')[0:3]) for x in predictions['true']]

In [131]:
predictions.head()

,true,predicted,predicted_3,true_3
0,4.2.1.20,4.2.1.20,4.2.1,4.2.1
1,2.3.1.54,2.3.1.54,2.3.1,2.3.1
2,1.1.1.27,1.1.1.27,1.1.1,1.1.1
3,1.7.1.13,1.7.1.13,1.7.1,1.7.1
4,1.4.1.1,2.8.4.3,2.8.4,1.4.1


In [132]:
matches = ones[['true_3', 'predicted_3']].assign(NE=ones.true != ones.predicted)['NE']
print('accuracy for EC1 at level 3:',1-np.sum(matches)/len(ones))
pmatches = predictions[['true_3', 'predicted_3']].assign(NE=predictions.true != predictions.predicted)['NE']
print('accuracy overall at level 3:',1-np.sum(pmatches)/len(predictions))

accuracy for EC1 at level 3: 0.8380439442334515
accuracy overall at level 3: 0.862714


In [134]:
len(Counter(predictions['predicted_3']))

153

In [135]:
len(Counter(ones['predicted_3']))

56

In [136]:
predictions.to_csv('mifaser_1Mpredictions_012420.csv',index=False)
ones.to_csv('mifaserEC1_1Mpredictions_012420.csv',index=False)

In [164]:
true_count = pd.DataFrame.from_dict(Counter(ones['true_3']),orient='index').reset_index()
true_count.columns = ['EC','count_true']
pred_count = pd.DataFrame.from_dict(Counter(ones['predicted_3']),orient='index').reset_index()
pred_count.columns=['EC','count_predicted']
onescount = true_count.merge(pred_count,on='EC',how='outer')
filtered = onescount[onescount['count_true']>=5]
filtered = filtered[filtered['count_predicted']>=5]
filtered = filtered.fillna(value=0)

In [165]:
filtered

,EC,count_true,count_predicted
0,1.1.1,36187,38591.0
1,1.7.1,774,847.0
2,1.3.1,5084,5523.0
3,1.5.1,1060,954.0
4,1.17.1,2579,2697.0
5,1.97.1,6280,7222.0
6,1.4.1,8457,9295.0
7,1.6.1,4547,4897.0
8,1.2.1,14757,15667.0
9,1.8.1,8528,9926.0


In [166]:
filtered.to_csv("preds0120_EC1_counts.csv",index=False)

In [167]:
true_count = pd.DataFrame.from_dict(Counter(predictions['true_3']),orient='index').reset_index()
true_count.columns = ['EC','count_true']
pred_count = pd.DataFrame.from_dict(Counter(predictions['predicted_3']),orient='index').reset_index()
pred_count.columns=['EC','count_predicted']
onescount = true_count.merge(pred_count,on='EC',how='outer')
filtered = onescount[onescount['count_true']>=5]
filtered = filtered[filtered['count_predicted']>=5]
filtered = filtered.fillna(value=0)

In [168]:
filtered

,EC,count_true,count_predicted
0,4.2.1,50232,49943.0
1,2.3.1,30496,30442.0
2,1.1.1,40220,38591.0
3,1.7.1,939,847.0
4,1.4.1,9222,9295.0
5,2.8.4,4038,3696.0
6,2.7.7,77569,83227.0
7,6.3.5,22171,24414.0
8,5.4.99,10120,9666.0
9,2.5.1,30464,29974.0


In [169]:
filtered.to_csv("preds0120_overall_counts.csv",index=False)